In [1]:
from ltr.client import ElasticClient
client = ElasticClient()

In [6]:
from ltr import download
download();

data/tmdb.json already exists
data/blog.jsonl already exists
data/osc_judgments.txt already exists
data/RankyMcRankFace.jar already exists
data/title_judgments.txt already exists
GET http://es-learn-to-rank.labs.o19s.com/genome_judgments.txt
data/sample_judgments_train.txt already exists
Done.


In [ ]:
from ltr.index import rebuild_tmdb
rebuild_tmdb(client)

Here I've used all 40 queries from the title judgements and aprox. 45 queries from the genome data set (starting from the 41st query in the list).

I've used the title, overview, genre, and tagline. 

The results themselves seem to be less about Berlin itself, but rather having their plot in Berlin.
Based on our users and use cases this could be either performing better or worse. *Are the user looking for recommendations or specific movies?*
To answer that we'd need a much bigger sample set of random queries from a single source during a specific time period in order to optimize in one way or the other.

With such a sample I'd probably try with a function score feature with the **release_year** and **votes**, to test out whether users prefer rather newer and/or better rated movies.


In [2]:
config = {"validation": {
              "index": "tmdb",
              "params": {
                  "keywords": "rambo"
              }    
           },
           "featureset": {
            "features": [
                { #1
                    "name": "title_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"title": "{{keywords}}"}
                    }
                },
                { #2
                    "name": "overview_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"overview": "{{keywords}}"}
                    }
                },
                { #3
                    "name": "tagline_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"tagline": "{{keywords}}"}
                    }
                },
                { #4
                    "name": "genre_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"genres": "{{keywords}}"}
                    }
                }
            ]
    }}

from ltr import setup
setup(client, config=config, index='tmdb', featureset='zenit-movies-combined')

Removed Default LTR feature store [Status: 200]
Initialize Default LTR feature store [Status: 200]
Create zenit-movies-combined feature set [Status: 201]


In [7]:
from ltr.log import judgments_to_training_set
trainingSet = judgments_to_training_set(client, 
                                        judgmentInFile='data/combined_title_and_genome_judgments.csv', 
                                        trainingOutFile='data/combined_title_and_genome_judgments_train.txt', 
                                        featureSet='zenit-movies-combined')

Recognizing 89 queries...
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for rambo (0/89)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for rocky (1/89)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for war games (2/89)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for crocodile dundee (3/89)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for matrix (4/89)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for contact (5/89)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for space jam (6/89)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for battlestar galactica (7/89)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for her (8/89)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for jobs (9/89)


Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for amy smart (57/89)
Missing doc 253941
Missing doc 225130
Missing doc 13716
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for android(s)/cyborg(s) (58/89)
Missing doc 133252
Missing doc 253768
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for androids (59/89)
Missing doc 206216
Missing doc 94174
Missing doc 58423
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for animal movie (60/89)
Missing doc 13057
Missing 

In [8]:
from ltr import train
trainLog  = train(client,
                  trainingInFile='data/combined_title_and_genome_judgments_train.txt',
                  index='tmdb',
                  featureSet='zenit-movies-combined',
                  modelName='zenit-genome-title-combined')

Running java -jar data/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t DCG@10 -tree 50 -bag 1 -leaf 10 -frate 1.0 -srate 1.0 -train data/combined_title_and_genome_judgments_train.txt -save data/zenit-genome-title-combined_model.txt 
DONE
Delete model zenit-genome-title-combined: 404
Created Model zenit-genome-title-combined [Status: 201]


In [10]:
from ltr import search
search(client, "berlin", modelName='zenit-genome-title-combined')

{"size": 5, "query": {"sltr": {"params": {"keywords": "berlin"}, "model": "zenit-genome-title-combined"}}}
Searching tmdb - {'size': 5, 'query': [Status: 200]
Victoria 
3.5443358 
2015 
['Drama', 'Romance', 'Thriller'] 
A young Spanish woman who has newly moved to Berlin finds her flirtation with a local guy turn potentially deadly as their night out with his friends reveals a dangerous secret. 
---------------------------------------
Downfall 
3.5443358 
2004 
['Drama', 'History', 'War'] 
In April of 1945, Germany stands at the brink of defeat with the Russian Army closing in from the east and the Allied Expeditionary Force attacking from the west. In Berlin, capital of the Third Reich, Adolf Hitler proclaims that Germany will still achieve victory and orders his generals and advisers to fight to the last man. When the end finally does come, and Hitler lies dead by his own hand, what is left of his military must find a way to end the killing that is the Battle of Berlin, and lay down 